**Cái này nó khác file AStar_new 1 tí ở giao diện. File AStar_new khi set map bự (rows: 50, cols: 50 thì nó không show full map)**

In [6]:
import random
import math
import heapq
import tkinter as tk
from tkinter import ttk, messagebox
from dataclasses import dataclass
from typing import List, Optional

In [7]:
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y
class Cell:
    def __init__(self, position, parent=None, status='free', g=0, h=0, f=0):
        self.position = position   
        self.parent = parent       
        self.status = status       # {'free', 'start', 'dirty', 'clean'}

        # các chi phí
        self.g = g   
        self.h = h  
        self.f = f   
        # cphí làm sạch ( = 1, tăng thêm sau mỗi hành động)
        self.cleaning_cost = 1
    def move_cost(self):
        return 1
    def update_f(self):
        self.f = self.g + self.h

    def __eq__(self, other):
        return (self.position.x == other.position.x and self.position.y == other.position.y)

    def __repr__(self):
        return (f"Cell({self.position.x}, {self.position.y}, "
                f"status={self.status}, g={self.g}, h={self.h}, f={self.f}, "
                f"clean_cost={self.cleaning_cost})")

    # def calculate_h(self, goal):
    #     self.h = math.sqrt((self.position.x - goal.position.x)**2 + (self.position.y - goal.position.y)**2)
    #     self.update_f()
    
    def increase_cleaning_cost(self):
        if self.status == 'dirty':
            self.cleaning_cost += 1

    def apply_cleaning_cost(self):
        if self.status == 'dirty':
            # self.g += self.cleaning_cost
            # self.update_f()
            self.status = 'clean'

In [8]:
class RobotCleaner:
    def __init__(self, rows, cols, num_dirty, num_obstacles):
        self.rows = rows
        self.cols = cols
        self.num_dirty = num_dirty
        self.num_obstacles = num_obstacles  # số lượng chướng ngại vật do người dùng nhập

        self.grid = []
        self.robot_pos = None
        self.dirty_cells = []

        self.total_move_cost = 0
        self.total_cleaning_cost = 0  # tổng chi phí tăng dần của các ô bẩn
        self.initialize_grid()

    def initialize_grid(self):
        self.grid = [[Cell(Position(x, y)) for y in range(self.cols)] for x in range(self.rows)]

     
        rx, ry = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
        self.robot_pos = Position(rx, ry)
        self.grid[rx][ry].status = 'start'

        # Đặt chướng ngại vật
        count = 0
        while count < self.num_obstacles:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'obstacle'
                count += 1

        # Đặt ô bẩn ngẫu nhiên
        count = 0
        while count < self.num_dirty:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'dirty'
                self.dirty_cells.append(self.grid[x][y])
                count += 1

    def heuristic(self, pos1, pos2):
        # Chebyshev (8 hướng)
        return max(abs(pos1.x - pos2.x), abs(pos1.y - pos2.y))

    def get_neighbors(self, cell):
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),           (0, 1),
            (1, -1),  (1, 0),  (1, 1)
        ]
        neighbors = []
        for dx, dy in directions:
            nx, ny = cell.position.x + dx, cell.position.y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols:
                n = self.grid[nx][ny]
                if n.status != 'obstacle':  # tránh vật cản
                    neighbors.append(n)
        return neighbors

    def reset_cells(self):
        for x in range(self.rows):
            for y in range(self.cols):
                c = self.grid[x][y]
                c.g = c.h = c.f = 0
                c.parent = None

    def find_path(self, start, goal):
        self.reset_cells()
        start_cell = self.grid[start.x][start.y]
        goal_cell = self.grid[goal.x][goal.y]

        open_list = []
        closed = set()
        counter = 0

        start_cell.g = 0
        start_cell.h = self.heuristic(start, goal)
        start_cell.f = start_cell.g + start_cell.h
        heapq.heappush(open_list, (start_cell.f, counter, start_cell))

        while open_list:
            f, _, current = heapq.heappop(open_list)

            if current.position.x == goal.x and current.position.y == goal.y:
                path = []
                while current:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]

            closed.add((current.position.x, current.position.y))

            for neighbor in self.get_neighbors(current):
                if (neighbor.position.x, neighbor.position.y) in closed:
                    continue

                tentative_g = current.g + neighbor.move_cost()
                if neighbor.parent is None or tentative_g < neighbor.g:
                    neighbor.parent = current
                    neighbor.g = tentative_g
                    neighbor.h = self.heuristic(neighbor.position, goal)
                    neighbor.update_f()
                    counter += 1
                    heapq.heappush(open_list, (neighbor.f, counter, neighbor))
        return None

    def print_map(self, current_robot_pos=None):
        rp = current_robot_pos if current_robot_pos else self.robot_pos
        for x in range(self.rows):
            row = []
            for y in range(self.cols):
                cell = self.grid[x][y]
                if rp.x == x and rp.y == y:
                    row.append("R")
                elif cell.status == "dirty":
                    row.append("D")
                elif cell.status == "clean":
                    row.append("C")
                elif cell.status == "start":
                    row.append("S")
                elif cell.status == "obstacle":
                    row.append("O")
                else:
                    row.append(".")
            print(" ".join(row))
        print()

    def clean_all(self):
        current_pos = self.robot_pos
        print("Bản đồ ban đầu:")
        self.print_map(current_pos)
        print('-----------------------------------------')

        while self.dirty_cells:
            nearest = min(self.dirty_cells, key=lambda d: self.heuristic(current_pos, d.position))
            path = self.find_path(current_pos, nearest.position)
            if not path:
                print(f" Không tìm được đường đến ô ({nearest.position.x}, {nearest.position.y}) - bị chặn bởi chướng ngại vật")
                self.dirty_cells.remove(nearest)
                continue

            for step_pos in path[1:]:
                self.total_move_cost += 1
                for cell in self.dirty_cells:
                    cell.increase_cleaning_cost()
                current_pos = step_pos
                self.print_map(current_pos)

            nearest.apply_cleaning_cost()
            self.total_cleaning_cost += nearest.cleaning_cost
            self.dirty_cells.remove(nearest)
            current_pos = nearest.position
            self.robot_pos = current_pos

            for cell in self.dirty_cells:
                cell.increase_cleaning_cost()

            print(f"Hút bụi tại ô ({nearest.position.x}, {nearest.position.y})")
            self.print_map(current_pos)
            print(f"Chi phí di chuyển hiện tại: {self.total_move_cost}")
            print(f"Tổng chi phí làm sạch cộng dồn: {self.total_cleaning_cost}")
            print('-----------------------------------------')

        print("Hoàn thành làm sạch tất cả ô!")
        print(f"Chi phí di chuyển: {self.total_move_cost}")
        print(f"Chi phí làm sạch: {self.total_cleaning_cost}")
        print(f"Tổng chi phí cuối cùng: {self.total_move_cost + self.total_cleaning_cost}")

In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
from typing import List, Optional

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Robot Cleaner — A*")
        self.resizable(False, False)

        # ---- Controls ----
        self.rows_var  = tk.IntVar(value=8)
        self.cols_var  = tk.IntVar(value=10)
        self.dirty_var = tk.IntVar(value=6)
        self.obst_enable_var = tk.BooleanVar(value=True)    # bật/tắt obstacles
        self.obst_var  = tk.IntVar(value=10)                # số chướng ngại vật
        self.seed_var  = tk.IntVar(value=42)
        self.delay_var = tk.IntVar(value=250)               # ms giữa các bước

        self.columnconfigure(0, weight=1)
        self.rowconfigure(2, weight=1)

        setup = ttk.LabelFrame(self, text="Setup", padding=10)
        setup.grid(row=0, column=0, sticky="ew", padx=10, pady=(10,6))
        for c in range(13):
            setup.columnconfigure(c, weight=0)
        setup.columnconfigure(9, weight=1)

        ttk.Label(setup, text="Rows").grid(row=0, column=0, padx=(0,4), sticky="w")
        ttk.Spinbox(setup, from_=3, to=200, textvariable=self.rows_var, width=5)\
            .grid(row=0, column=1, padx=(0,10))

        ttk.Label(setup, text="Cols").grid(row=0, column=2, padx=(0,4))
        ttk.Spinbox(setup, from_=3, to=200, textvariable=self.cols_var, width=5)\
            .grid(row=0, column=3, padx=(0,10))

        ttk.Label(setup, text="Dirty").grid(row=0, column=4, padx=(0,4))
        ttk.Spinbox(setup, from_=1, to=2000, textvariable=self.dirty_var, width=6)\
            .grid(row=0, column=5, padx=(0,10))

        ttk.Label(setup, text="Seed").grid(row=0, column=6, padx=(0,4))
        ttk.Entry(setup, textvariable=self.seed_var, width=8)\
            .grid(row=0, column=7, padx=(0,10))

        ttk.Label(setup, text="Delay (ms)").grid(row=0, column=8, padx=(0,6))
        ttk.Scale(setup, from_=0, to=1500, variable=self.delay_var, orient="horizontal")\
            .grid(row=0, column=9, sticky="ew", padx=(0,10))

        ttk.Label(setup, text="Obstacles").grid(row=0, column=10, padx=(0,4))
        self.obst_spin = ttk.Spinbox(setup, from_=0, to=5000, textvariable=self.obst_var, width=7)
        self.obst_spin.grid(row=0, column=11, padx=(0,10))

        self.obst_chk = ttk.Checkbutton(
            setup, text="Use obstacles",
            variable=self.obst_enable_var,
            command=self._toggle_obstacles_enabled
        )
        self.obst_chk.grid(row=0, column=12, padx=(0,0))

        # Buttons
        btns = ttk.Frame(self, padding=(10,0,10,10))
        btns.grid(row=1, column=0, sticky="ew")
        self.run_btn   = ttk.Button(btns, text="Generate & Run", command=self.start_run)
        self.pause_btn = ttk.Button(btns, text="Pause", command=self.toggle_pause, state="disabled")
        self.stop_btn  = ttk.Button(btns, text="Stop", command=self.stop_run, state="disabled")
        self.zoom_in_btn  = ttk.Button(btns, text="Zoom +", command=lambda: self.zoom(1.25))
        self.zoom_out_btn = ttk.Button(btns, text="Zoom −", command=lambda: self.zoom(0.8))
        self.fit_btn      = ttk.Button(btns, text="Fit",    command=self.fit_to_view)
        self.run_btn.grid(row=0, column=0, padx=3)
        self.pause_btn.grid(row=0, column=1, padx=3)
        self.stop_btn.grid(row=0, column=2, padx=3)
        self.zoom_in_btn.grid(row=0, column=3, padx=(12,3))
        self.zoom_out_btn.grid(row=0, column=4, padx=3)
        self.fit_btn.grid(row=0, column=5, padx=3)

        # Layout chính
        main = ttk.Frame(self, padding=10)
        main.grid(row=2, column=0, sticky="nsew")
        main.columnconfigure(0, weight=1)
        main.rowconfigure(0, weight=1)

        # --- Map frame with scrollbars ---
        map_frame = ttk.LabelFrame(main, text="Map", padding=8)
        map_frame.grid(row=0, column=0, sticky="nsew", padx=(0,10))
        map_frame.columnconfigure(0, weight=1)
        map_frame.rowconfigure(0, weight=1)

        canvas_wrap = ttk.Frame(map_frame)
        canvas_wrap.grid(row=0, column=0, sticky="nsew")
        canvas_wrap.columnconfigure(0, weight=1)
        canvas_wrap.rowconfigure(0, weight=1)

        self.cell_size = 26
        self.viewport_w = 800
        self.viewport_h = 600
        self.canvas = tk.Canvas(
            canvas_wrap,
            width=self.viewport_w,
            height=self.viewport_h,
            bg="#ffffff",
            highlightthickness=1,
            highlightbackground="#bbb"
        )
        self.canvas.grid(row=0, column=0, sticky="nsew")

        self.vbar = ttk.Scrollbar(canvas_wrap, orient="vertical", command=self.canvas.yview)
        self.hbar = ttk.Scrollbar(canvas_wrap, orient="horizontal", command=self.canvas.xview)
        self.vbar.grid(row=0, column=1, sticky="ns")
        self.hbar.grid(row=1, column=0, sticky="ew")
        self.canvas.configure(yscrollcommand=self.vbar.set, xscrollcommand=self.hbar.set)

        # Mouse wheel scroll
        def _on_mousewheel(event):
            step = -1 if event.delta > 0 else 1
            if event.state & 0x0001:  # Shift => cuộn ngang
                self.canvas.xview_scroll(step, "units")
            else:
                self.canvas.yview_scroll(step, "units")
        self.canvas.bind("<MouseWheel>", _on_mousewheel)
        self.canvas.bind("<Shift-MouseWheel>", _on_mousewheel)
        self.canvas.bind("<Button-4>", lambda e: self.canvas.yview_scroll(-1, "units"))  # Linux
        self.canvas.bind("<Button-5>", lambda e: self.canvas.yview_scroll( 1, "units"))

        # Bảng thông tin
        side = ttk.Frame(main, padding=0)
        side.grid(row=0, column=1, sticky="n")

        costs = ttk.LabelFrame(side, text="Live Costs", padding=10)
        costs.grid(row=0, column=0, sticky="ew")
        self.step_lbl  = ttk.Label(costs, text="Step: 0");       self.step_lbl.grid(row=0, column=0, sticky="w")
        self.dleft_lbl = ttk.Label(costs, text="Dirty left: 0"); self.dleft_lbl.grid(row=1, column=0, sticky="w")
        self.move_lbl  = ttk.Label(costs, text="Move: 0");       self.move_lbl.grid(row=2, column=0, sticky="w")
        self.delay_lbl = ttk.Label(costs, text="Delay: 0");      self.delay_lbl.grid(row=3, column=0, sticky="w")
        self.total_lbl = ttk.Label(costs, text="Total: 0", font=("Segoe UI", 10, "bold"))
        self.total_lbl.grid(row=4, column=0, sticky="w")

        self.show_path_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(costs, text="Show path", variable=self.show_path_var,
                        command=self.draw_path_once).grid(row=5, column=0, sticky="w", pady=(6,0))

        ttk.Separator(side, orient="horizontal").grid(row=1, column=0, sticky="ew", pady=10)

        legend = ttk.LabelFrame(side, text="Color Legend", padding=10)
        legend.grid(row=2, column=0, sticky="ew")
        color_items = [
            ("#222222", "Robot (R)"),
            ("#6ab8ff", "Start"),
            ("#ff6b6b", "Dirty"),
            ("#6bd66b", "Clean"),
            ("#f4a261", "Obstacle"),
            ("#eeeeee", "Free"),
        ]
        for i, (color, label) in enumerate(color_items):
            row = ttk.Frame(legend)
            row.grid(row=i, column=0, sticky="w", pady=2)
            box = tk.Canvas(row, width=18, height=18, bg=color,
                            highlightthickness=1, highlightbackground="#666")
            box.grid(row=0, column=0, padx=(0,6))
            ttk.Label(row, text=label).grid(row=0, column=1, sticky="w")

        # Runtime state
        self.rc: Optional['RobotCleaner'] = None
        self.running = False
        self.paused = False
        self.after_id = None

        self.step_no = 0
        self.move_cost = 0
        self.delay_cost = 0
        self.current_path: List['Position'] = []
        self.path_idx = 1

        self.colors = {
            'free':     "#eeeeee",
            'dirty':    "#ff6b6b",
            'clean':    "#6bd66b",
            'start':    "#6ab8ff",
            'robot':    "#222222",
            'path':     "#ffd166",
            'obstacle': "#f4a261",
        }

        self.cell_rect: List[List[Optional[int]]] = []
        self.robot_id: Optional[int] = None
        self.robot_text_id: Optional[int] = None
        self.path_item_id: Optional[int] = None

        # set trạng thái ban đầu cho obstacles spinbox
        self._toggle_obstacles_enabled()

    # ----- Canvas helpers -----
    def init_grid_canvas(self):
        # Vẽ toàn bộ theo cell_size hiện tại
        self._redraw_all()

    def _redraw_all(self):
        if not self.rc: return
        rows, cols = self.rc.rows, self.rc.cols
        s = self.cell_size

        self.canvas.delete("all")
        self.cell_rect = [[None]*cols for _ in range(rows)]

        for i in range(rows):
            for j in range(cols):
                x0, y0 = j*s, i*s
                x1, y1 = x0+s, y0+s
                st = self.rc.grid[i][j].status
                fill = self.colors.get(st, "#eeeeee")
                rid = self.canvas.create_rectangle(
                    x0, y0, x1, y1, fill=fill,
                    outline="#666" if st == "obstacle" else "#999"
                )
                self.cell_rect[i][j] = rid

        # robot
        rx, ry = self.rc.robot_pos.x, self.rc.robot_pos.y
        x0, y0 = ry*s, rx*s
        x1, y1 = x0+s, y0+s
        self.robot_id = self.canvas.create_oval(x0+3, y0+3, x1-3, y1-3,
                                                fill=self.colors['robot'], outline="#ffffff", width=2)
        self.robot_text_id = self.canvas.create_text((x0+x1)//2, (y0+y1)//2,
                                                     text="R", fill="#fff", font=("Arial", 10, "bold"))

        # path
        self.path_item_id = None
        self.draw_path_once()

        # scroll region bao hết map
        self.canvas.config(scrollregion=(0, 0, cols*s, rows*s))

    def update_cell_color(self, i: int, j: int, status: str):
        rid = self.cell_rect[i][j]
        if rid is not None:
            self.canvas.itemconfig(rid, fill=self.colors.get(status, "#eeeeee"),
                                   outline="#666" if status == "obstacle" else "#999")

    def move_robot_to(self, pos: 'Position'):
        s = self.cell_size
        x0, y0 = pos.y*s, pos.x*s
        x1, y1 = x0+s, y0+s
        if self.robot_id is not None:
            self.canvas.coords(self.robot_id, x0+3, y0+3, x1-3, y1-3)
        if self.robot_text_id is not None:
            self.canvas.coords(self.robot_text_id, (x0+x1)//2, (y0+y1)//2)

    def _clear_path(self):
        if self.path_item_id:
            self.canvas.delete(self.path_item_id)
            self.path_item_id = None

    def draw_path_once(self):
        if not self.show_path_var.get():
            self._clear_path(); return
        if not (self.rc and self.current_path) or self.path_idx >= len(self.current_path):
            self._clear_path(); return

        s = self.cell_size
        rx, ry = self.rc.robot_pos.x, self.rc.robot_pos.y
        pts = [ry*s + s/2, rx*s + s/2]
        steps = self.current_path[self.path_idx:self.path_idx+5]
        for p in steps:
            pts.extend([p.y*s + s/2, p.x*s + s/2])
        if len(pts) < 4 and self.path_idx < len(self.current_path):
            g = self.current_path[-1]
            pts.extend([g.y*s + s/2, g.x*s + s/2])
        if len(pts) < 4:
            self._clear_path(); return

        try:
            if self.path_item_id:
                self.canvas.coords(self.path_item_id, *pts)
            else:
                self.path_item_id = self.canvas.create_line(*pts, width=2, fill=self.colors['path'])
        except tk.TclError:
            self._clear_path()

    # ----- Side info -----
    def update_side(self):
        self.step_lbl.config(text=f"Step: {self.step_no}")
        dleft = len(self.rc.dirty_cells) if self.rc else 0
        self.dleft_lbl.config(text=f"Dirty left: {dleft}")
        self.move_lbl.config(text=f"Move: {int(self.move_cost)}")
        self.delay_lbl.config(text=f"Delay: {int(self.delay_cost)}")
        self.total_lbl.config(text=f"Total: {int(self.move_cost + self.delay_cost)}")

    # ----- Run control -----
    def start_run(self):
        if self.running: return
        try:
            import random
            random.seed(self.seed_var.get())
            num_obstacles = self.obst_var.get() if self.obst_enable_var.get() else 0

            self.rc = RobotCleaner(
                self.rows_var.get(),
                self.cols_var.get(),
                self.dirty_var.get(),
                num_obstacles
            )
            if not hasattr(self.rc, "move_cost_per_step"):
                self.rc.move_cost_per_step = 1
        except Exception as e:
            messagebox.showerror("Error", str(e)); return

        self.running = True
        self.paused = False
        self.step_no = 0
        self.move_cost = 0
        self.delay_cost = 0
        self.current_path = []
        self.path_idx = 1

        self.run_btn.config(state="disabled")
        self.pause_btn.config(text="Pause", state="normal")
        self.stop_btn.config(state="normal")

        # khoá obstacle controls khi đang chạy
        self.obst_chk.config(state="disabled")
        self.obst_spin.config(state="disabled")

        self.init_grid_canvas()
        # tự fit khung nhìn
        self.fit_to_view()
        self.update_side()
        self.schedule_tick(50)

    def toggle_pause(self):
        if not self.running: return
        self.paused = not self.paused
        self.pause_btn.config(text="Resume" if self.paused else "Pause")

    def stop_run(self):
        self.running = False
        self.paused = False
        self.run_btn.config(state="normal")
        self.pause_btn.config(text="Pause", state="disabled")
        self.stop_btn.config(state="disabled")
        if self.after_id:
            self.after_cancel(self.after_id); self.after_id = None

        # mở khoá obstacle controls
        self.obst_chk.config(state="normal")
        self._toggle_obstacles_enabled()

    def _toggle_obstacles_enabled(self):
        state = "normal" if self.obst_enable_var.get() else "disabled"
        self.obst_spin.config(state=state)

    def schedule_tick(self, ms: int):
        self.after_id = self.after(ms, self.tick)

    def tick(self):
        if not self.running or not self.rc:
            return

        if self.paused:
            self.schedule_tick(100); return

        if not self.rc.dirty_cells:
            self.stop_run(); return

        # Nếu chưa có path tới mục tiêu, tìm tới dirty gần nhất theo heuristic
        if not self.current_path:
            cur = self.rc.robot_pos
            target_cell = min(self.rc.dirty_cells, key=lambda c: self.rc.heuristic(cur, c.position))
            path = self.rc.find_path(cur, target_cell.position)
            if not path:
                # không có đường đi -> bỏ ô này rồi tiếp tục
                self.rc.dirty_cells.remove(target_cell)
                self.schedule_tick(20)
                return
            self.current_path = path
            self.path_idx = 1
            self.draw_path_once()

        # Đi 1 bước
        if self.path_idx < len(self.current_path):
            nxt = self.current_path[self.path_idx]

            # cost mô phỏng
            self.move_cost += self.rc.move_cost_per_step
            self.delay_cost += len(self.rc.dirty_cells)

            # di chuyển robot
            self.rc.robot_pos = nxt
            self.move_robot_to(self.rc.robot_pos)
            self.step_no += 1
            self.path_idx += 1

            # nếu tới đích: làm sạch
            if self.path_idx == len(self.current_path):
                target_pos = self.current_path[-1]
                self.rc.grid[target_pos.x][target_pos.y].status = "clean"
                self.rc.dirty_cells = [
                    c for c in self.rc.dirty_cells
                    if not (c.position.x == target_pos.x and c.position.y == target_pos.y)
                ]
                self.update_cell_color(target_pos.x, target_pos.y, "clean")
                # “suck” cũng tính như 1 lần delay
                self.delay_cost += len(self.rc.dirty_cells)
                self.current_path = []
                self.path_idx = 1
                self.draw_path_once()
            else:
                self.draw_path_once()

            self.update_side()

        d = max(0, int(self.delay_var.get()))
        self.schedule_tick(d)

    # ----- Zoom / Fit -----
    def _redraw_after_resize(self):
        """Giữ robot & path đúng vị trí sau khi đổi cell_size."""
        self._redraw_all()

    def zoom(self, factor: float):
        old = self.cell_size
        new = int(max(4, min(80, old * factor)))   # giới hạn cell 4..80 px để mượt
        if new == old: return
        self.cell_size = new
        self._redraw_after_resize()

    def fit_to_view(self):
        if not self.rc: return
        rows, cols = self.rc.rows, self.rc.cols
        vp_w = int(self.canvas.winfo_width()  or self.viewport_w)
        vp_h = int(self.canvas.winfo_height() or self.viewport_h)
        if cols == 0 or rows == 0: return
        size_w = max(4, vp_w // cols)
        size_h = max(4, vp_h // rows)
        new = int(min(80, size_w, size_h))  # giới hạn max 80
        if new < 4: new = 4
        if new != self.cell_size:
            self.cell_size = new
            self._redraw_after_resize()


In [10]:
if __name__ == "__main__":
    app = App()
    app.mainloop()